In [1]:
import random
import time

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}
playing = True

In [2]:
# Create a Card Class

class Card:   
    def __init__(self,rank,suit):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
    
    def __str__(self):
        return '{} of {}'.format(self.rank,self.suit)

#Create a Deck Class
class Deck:    
    def __init__(self):
        self.deck = []  
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(rank,suit)) 
    
    def __str__(self):        
        deck_print = ''
        for i in self.deck:
            deck_print += str(i)+'\n'
        return deck_print
    
    def shuffle(self):
        random.shuffle(self.deck)
        return [i.__str__() for i in self.deck]

    def deal(self):
        single_card = self.deck.pop()
        return single_card.__str__(),single_card

# Create a Hand Class
class Hand:
    def __init__(self):
        self.cards = []  # start with an empty list as we did in the Deck class
        self.value = 0   # start with zero value
        self.aces = 0    # add an attribute to keep track of aces
    
    def add_card(self,card_value,card):
        self.cards.append(card_value)
        self.value += values[card.rank]
        if card.rank == 'Ace':
            self.aces += 1
        
    def adjust_for_ace(self):
        self.value -= 10
        self.aces -= 1
        
    def add_burst(self):
        self.cards.append('burst!')
        
class Chips:    
    def __init__(self):
        self.total = 100  # This can be set to a default value or supplied by a user input
        
    def win_bet(self,bet = 0):
        self.bet = bet
        self.total += self.bet
            
    def lose_bet(self,bet = 0):
        self.bet = bet
        self.total -= self.bet

In [3]:
# Funtions

# bet enter funtuin
def take_bet():
    while True:
        try:
            bet = ''
            while type(bet) != int:
                bet = int(input('Please enter your bet(int only) : '))
            return bet
            break
        except ValueError:
            print('Oops it\'s not a number.Please enter again')

def hit(deck,hand):
    
    control = True
    while control:
        
        # input yes or no
        while True:
            a = input('Hits or no(y/n)? ')
            if a == "y":
                break
            elif a == "n":
                control = False
                return False
                break
            else:
                pass
            
        # add card
        if a == 'y':
            next_card_value,next_card = deck.deal()
            hand.add_card(next_card_value,next_card)
            if hand.value > 21:
                hand.add_burst()
                return False
            return True
            break
            
# show cards
def show_some(player):
    n = 0
    l = []
    for i in player.cards:
        if n ==1:
            n += 1
            continue
        n += 1
        if i != 'burst!':
            l.append(i)
    print(l)
    return l
          
def show_all(player):
    return player.cards,player.value
    
def see_value(player):
    while True:
        print(show_all(player),end = '\r')
        time.sleep(2)
        print(' '*len(str(show_all(player))),end = '\r')
        check = input('check or not(y\\n)? ')
        if check == 'y':            
            break      

In [6]:
# main 
test_d = Deck()
test_d.shuffle()
#print(test_d.shuffle())

# money
player1_money = Chips()
player2_money = Chips()

while playing:
    # cards 
    player1 = Hand()
    player2 = Hand()
    
    for num in range(2):
        card = test_d.deal()
        card2 = test_d.deal()
        player1.add_card(card[0],card[1])
        player2.add_card(card2[0],card2[1])
    
    # bet decide
    bet = take_bet()
    
    con1,con2 = True,True
    con3 = False
    
    while True:

        while con1:
            print('Player1 goes ')
            print('Player2\'s open card : ',end = '')
            show_some(player2)
            see_value(player1)
            con1 = hit(test_d,player1)
            while player1.aces != 0:
                if 'burst!' in player1.cards and player1.aces != 0:
                    player1.adjust_for_ace()
                    con1 = True
                else:
                    break
            see_value(player1)

            if 'burst!' in player1.cards and player1.value > 21:
                print('Player1 burst!.Player2 wins!')
                player1_money.lose_bet(bet)
                player2_money.win_bet(bet)
                print('player1 now keeps $ {}.Player2 now keeps $ {}'.format(player1_money.total,player2_money.total))
                con3 = True
                con1,con2 = False,False
                break
            break

        while con2:   
            print('Player2 goes ')
            print('Player1\'s open card : ',end = '')
            show_some(player1)
            see_value(player2)
            con2 = hit(test_d,player2)
            while player2.aces != 0:
                if 'burst!' in player2.cards and player2.aces != 0:
                    player2.adjust_for_ace()
                    con2 = True
                else:
                    break
            see_value(player2)

            if 'burst!' in player2.cards and player2.value > 21:
                print('Player2 burst!.Player1 wins!')
                player2_money.lose_bet(bet)
                player1_money.win_bet(bet)
                print('player1 now keeps $ {}.Player2 now keeps $ {}'.format(player1_money.total,player2_money.total))
                con3 = True
                con1,con2 = False,False
                break
            break

        if con1 == con2 == con3 == False:   
            if player1.value > player2.value:
                print('Player1 wins!')
                player2_money.lose_bet(bet)
                player1_money.win_bet(bet)
                print('player1 now keeps $ {}.Player2 now keeps $ {}'.format(player1_money.total,player2_money.total))
                break

            elif player1.value < player2.value:
                print('Player2 wins!')
                player1_money.lose_bet(bet)
                player2_money.win_bet(bet)
                print('player1 now keeps $ {}.Player2 now keeps $ {}'.format(player1_money.total,player2_money.total))
                break

            elif player1.value == player2.value:
                print('tie')
                break
        elif con3:
            break
            
    while True:    
        pl = input("Wanna play again(y/n)?")
        if pl == 'n':
            playing = False
            break
        elif pl == 'y':
            playing = True
            break
            

Please enter your bet(int only) : 5
Player1 goes 
Player2's open card : ['Three of Diamonds']
check or not(y\n)? y                     
Hits or no(y/n)? y
check or not(y\n)? y                                        
Player2 goes 
Player1's open card : ['Four of Clubs', 'Queen of Spades']
check or not(y\n)? y                      
Hits or no(y/n)? y
check or not(y\n)? y                                         
Player1 goes 
Player2's open card : ['Three of Diamonds', 'Queen of Clubs']
check or not(y\n)? y                                        
Hits or no(y/n)? n
check or not(y\n)? y                                        
Player2 goes 
Player1's open card : ['Four of Clubs', 'Queen of Spades']
check or not(y\n)? y                                         
Hits or no(y/n)? y
check or not(y\n)? y                                                           
Player2 goes 
Player1's open card : ['Four of Clubs', 'Queen of Spades']
check or not(y\n)? y                                           